In [1]:
import os
import numpy as np
import tifffile as tif
import SimpleITK as sitk
import cv2
import sys
from scipy.ndimage import zoom

In [2]:


def load_memmap_arr(pth, mode="r", dtype = "uint16", shape = False):
    """
    Function to load memmaped array.

    Inputs
    -----------
    pth: path to array
    mode: (defaults to r)
    +------+-------------------------------------------------------------+
    | "r"  | Open existing file for reading only.                        |
    +------+-------------------------------------------------------------+
    | "r+" | Open existing file for reading and writing.                 |
    +------+-------------------------------------------------------------+
    | "w+" | Create or overwrite existing file for reading and writing.  |
    +------+-------------------------------------------------------------+
    | "c"  | Copy-on-write: assignments affect data in memory, but       |
    |      | changes are not saved to disk.  The file on disk is         |
    |      | read-only.                                                  |
    dtype: digit type
    shape: (tuple) shape when initializing the memory map array

    Returns
    -----------
    arr
    """
    if shape:
        assert mode =="w+", "Do not pass a shape input into this function unless initializing a new array"
        arr = np.lib.format.open_memmap(pth, dtype = dtype, mode = mode, shape = shape)
    else:
        arr = np.lib.format.open_memmap(pth, dtype = dtype, mode = mode)
    return arr

In [18]:
#seedname = "e106_f002_median"
#brainname = "t107_a235_median"


#seedname = "f110_k320_median"
#brainname = "c514_f003_median"

brainname = "taef_to_cffk"
seedname = "c514_f003_median_f110_k320_median_median"

In [25]:
seedname = "PRA_seed2"
brainnames = ["a235","c514","e106","f002","f003","k320","f110"]

In [30]:
src = "/home/emilyjanedennis/Desktop/vols"
seed = os.path.join(src,"tiffs/{}.tif".format(seedname))
parameterfld = "/home/emilyjanedennis/Desktop/ab2/"
parameters = [os.path.join(parameterfld,xx) for xx in os.listdir(parameterfld)]

output_fld = os.path.join(src,"out")
data_fld = os.path.join(src,"output_dirs")    
memmappth = os.path.join(output_fld,"to_PRAseed1_memmap.npy")
final_output_path = os.path.join(output_fld,"PRA_seed3.tif")


In [31]:
seed

'/home/emilyjanedennis/Desktop/vols/tiffs/PRA_seed2.tif'

In [32]:
vol = tif.imread(seed)
z,y,x = vol.shape
dtype = vol.dtype

arr = load_memmap_arr(memmappth,mode="w+",shape=(len(brainnames)+1,z,y,x),dtype=dtype)

arr[0]=tif.imread(seed)
arr.flush()
x=0

for brainname in brainnames:
    x=x+1
    brain = os.path.join(src,"output_dirs/{}_to_{}/result.2.tif".format(brainname,seedname))
    print(brainname)
    arr[x]=tif.imread(brain)
    arr.flush()

a235
c514
e106
f002
f003
k320
f110


In [33]:
vol=np.median(arr,axis=0)
tif.imsave(final_output_path,vol.astype(dtype))

In [34]:
final_output_path

'/home/emilyjanedennis/Desktop/vols/out/PRA_seed3.tif'

In [2]:
test = np.load('/home/emilyjanedennis/Desktop/forpeter_PRA-new-to_WHS/posttransformed_zyx_voxels.npy')

In [3]:
test #this is xyz in large WHS

array([[349.754098, 327.924628, 191.412998],
       [538.464435, 332.785502, 165.960599],
       [533.439624, 373.890042, 135.59669 ],
       [499.561842, 371.17776 , 205.462934],
       [554.936539, 375.736774, 232.753987],
       [437.978111, 399.184889, 145.232078],
       [441.519529, 401.72046 , 243.975854],
       [441.643486, 419.620244, 242.673456],
       [537.937724, 420.906876, 236.994834],
       [480.656416, 424.343765, 186.975619],
       [586.360133, 460.087783, 195.28692 ],
       [617.692932, 509.049266, 133.296668],
       [131.697403, 672.274387, 302.466235],
       [474.963753, 761.972308,  49.009447],
       [114.74703 , 807.544948, 289.354065],
       [711.27654 , 850.895152, 122.852196],
       [103.848113, 902.965575, 126.397039],
       [ 89.662891, 888.645971, 236.420107]])

In [4]:
test = np.load('/home/emilyjanedennis/Desktop/forpeter_PRA-new-to_WHS/posttransformed_zyx_voxels.npy')

# initialize
# in zyx
resample_dims = (455,774,302) #whs
original_dims = (865,1610,539) #largewhs
d1, d2 = test.shape
nx4centers = np.ones((d1, d2+1))
nx4centers[:, :-1] = test

# acount for resampling by creating transformmatrix
zr, yr, xr = resample_dims
z, y, x = original_dims

# apply scale diff
trnsfrmmatrix = np.identity(4)*(zr/float(z), yr/float(y), xr/float(x), 1)

# nx4 * 4x4 to give transform
trnsfmdpnts = nx4centers.dot(trnsfrmmatrix)  # z,y,x


In [5]:
nx4centers

array([[349.754098, 327.924628, 191.412998,   1.      ],
       [538.464435, 332.785502, 165.960599,   1.      ],
       [533.439624, 373.890042, 135.59669 ,   1.      ],
       [499.561842, 371.17776 , 205.462934,   1.      ],
       [554.936539, 375.736774, 232.753987,   1.      ],
       [437.978111, 399.184889, 145.232078,   1.      ],
       [441.519529, 401.72046 , 243.975854,   1.      ],
       [441.643486, 419.620244, 242.673456,   1.      ],
       [537.937724, 420.906876, 236.994834,   1.      ],
       [480.656416, 424.343765, 186.975619,   1.      ],
       [586.360133, 460.087783, 195.28692 ,   1.      ],
       [617.692932, 509.049266, 133.296668,   1.      ],
       [131.697403, 672.274387, 302.466235,   1.      ],
       [474.963753, 761.972308,  49.009447,   1.      ],
       [114.74703 , 807.544948, 289.354065,   1.      ],
       [711.27654 , 850.895152, 122.852196,   1.      ],
       [103.848113, 902.965575, 126.397039,   1.      ],
       [ 89.662891, 888.645971,

In [7]:
trnsfmdpnts

array([[183.97469895, 157.64823731, 107.24809906,   1.        ],
       [283.23851783, 159.98507984,  92.98720018,   1.        ],
       [280.59540916, 179.74589597,  75.97439774,   1.        ],
       [262.77530417, 178.44197903, 115.12023389,   1.        ],
       [291.90303497, 180.63370377, 130.41132481,   1.        ],
       [230.38154972, 191.90627583,  81.37307524,   1.        ],
       [232.24437653, 193.12523978, 136.6989015 ,   1.        ],
       [232.30957934, 201.73047755, 135.96917201,   1.        ],
       [282.96146176, 202.34901989, 132.78745801,   1.        ],
       [252.83083154, 204.00128827, 104.76184961,   1.        ],
       [308.43220869, 221.18505841, 109.41864534,   1.        ],
       [324.91362319, 244.72306328,  74.68570266,   1.        ],
       [ 69.27435649, 323.19277984, 169.4708775 ,   1.        ],
       [249.83642499, 366.31463751,  27.45983858,   1.        ],
       [ 60.35826434, 388.2234719 , 162.12417   ,   1.        ],
       [374.13968289, 409

In [3]:
#coords in [z,y,x], sets
# note if you made these measurements in an un-zoomed volume
# but in aligning you used a zoomed volume, you need
# to 'zoom' these points
x=np.array([[276,186,140],[373,189,127],[373,212,104],[353,219,154],[381,222,174],[323,234,114],[323,240,178],[323,251,177],[373,251,177],[344,252,142],[399,275,150],[416,308,106],[161,436,204],[351,492,36],[153,538,194],[469,560,100],[157,596,72],[145,596,154]])
np.save('/home/emilyjanedennis/Desktop/whs_forpeter.npy',x)

In [17]:
def fast_scandir(dirname):
    """ gets all folders recursively """
    subfolders = [f.path for f in os.scandir(dirname) if f.is_dir()]
    for dirname in list(subfolders):
        subfolders.extend(fast_scandir(dirname))
    return subfolders

def get_folderstructure(dirname):
    folderstructure = []
    for i in os.walk(dirname):
        folderstructure.append(i)
    return folderstructure

def get_onlyfolders(dirname):
    folderstructure = []
    for i in os.walk(dirname):
        if i.is_dir():
            folderstructure.append(i)
    return folderstructure

In [3]:
pwd

'/home/emilyjanedennis/Desktop/GitHub/rat_BrainPipe/tools/analysis'

In [25]:
x=fast_scandir("/home/emilyjanedennis/LightSheetData/lightserv/pbibawi/pb_udisco_647_488_4x/pb_udisco_647_488_4x-004/imaging_request_1/rawdata/resolution_3.6x/Ex_642_Em_2/corrected/")

/home/emilyjanedennis/LightSheetData/lightserv/pbibawi/pb_udisco_647_488_4x/pb_udisco_647_488_4x-004/imaging_request_1/rawdata/resolution_3.6x/Ex_642_Em_2/corrected/050940
/home/emilyjanedennis/LightSheetData/lightserv/pbibawi/pb_udisco_647_488_4x/pb_udisco_647_488_4x-004/imaging_request_1/rawdata/resolution_3.6x/Ex_642_Em_2/corrected/050940/050940_090361


In [26]:
len(x)

2

/jukebox/LightSheetData/lightserv/pbibawi/pb_udisco_647_488_4x/
    
    pb_udisco_647_488_4x-001/
      
      imaging_request_1/
         
         rawdata
             
             resolution*
                
                in resolution, there should be 1+ folders. Each of these is a channel
                    
                    488, 64*
                        
                        corrected (there are lots of other folders)
                             
                             tiffs

In [64]:

def get_folderstructure(dirname):
    folderstructure = []
    for i in os.walk(src):
        folderstructure.append(i)
    return folderstructure

src="/Users/emilydennis/Desktop/test"

for directory, subdirectories, files in get_folderstructure(src):
    if "rawdata" in directory:
        for ls in subdirectories:
            if 'resolution' in ls:
                rawdata=os.path.join(directory,ls)

for i in os.listdir(rawdata):
    if '488' in i:
        for j in os.listdir(os.path.join(rawdata,i)):
            if 'corrected' in j:
                reg_ch = os.path.join(rawdata,i,j)
    if '64' in i:
        for j in os.listdir(os.path.join(rawdata,i)):
            if 'corrected' in j:
                cell_ch = os.path.join(rawdata,i,j)

'/Users/emilydennis/Desktop/test/rawdata/resolution_10/xx_642_ch2/corrected'

In [22]:
x

['/Users/emilydennis/Desktop/test/a',
 '/Users/emilydennis/Desktop/test/rawdata',
 '/Users/emilydennis/Desktop/test/rawdata/resolution_10',
 '/Users/emilydennis/Desktop/test/rawdata/b',
 '/Users/emilydennis/Desktop/test/rawdata/resolution_10/xx_642_ch2',
 '/Users/emilydennis/Desktop/test/rawdata/resolution_10/ch1_488_xx',
 '/Users/emilydennis/Desktop/test/rawdata/resolution_10/xx_642_ch2/killerbee',
 '/Users/emilydennis/Desktop/test/rawdata/resolution_10/xx_642_ch2/naruto',
 '/Users/emilydennis/Desktop/test/rawdata/resolution_10/xx_642_ch2/corrected',
 '/Users/emilydennis/Desktop/test/rawdata/resolution_10/xx_642_ch2/killerbee/not',
 '/Users/emilydennis/Desktop/test/rawdata/resolution_10/xx_642_ch2/killerbee/other',
 '/Users/emilydennis/Desktop/test/rawdata/resolution_10/xx_642_ch2/killerbee/good',
 '/Users/emilydennis/Desktop/test/rawdata/resolution_10/xx_642_ch2/naruto/not',
 '/Users/emilydennis/Desktop/test/rawdata/resolution_10/xx_642_ch2/naruto/other',
 '/Users/emilydennis/Desktop

In [23]:
for dirname in x:
    if "resolution" in dirname:
        print(dirname)

/Users/emilydennis/Desktop/test/rawdata/resolution_10
/Users/emilydennis/Desktop/test/rawdata/resolution_10/xx_642_ch2
/Users/emilydennis/Desktop/test/rawdata/resolution_10/ch1_488_xx
/Users/emilydennis/Desktop/test/rawdata/resolution_10/xx_642_ch2/killerbee
/Users/emilydennis/Desktop/test/rawdata/resolution_10/xx_642_ch2/naruto
/Users/emilydennis/Desktop/test/rawdata/resolution_10/xx_642_ch2/corrected
/Users/emilydennis/Desktop/test/rawdata/resolution_10/xx_642_ch2/killerbee/not
/Users/emilydennis/Desktop/test/rawdata/resolution_10/xx_642_ch2/killerbee/other
/Users/emilydennis/Desktop/test/rawdata/resolution_10/xx_642_ch2/killerbee/good
/Users/emilydennis/Desktop/test/rawdata/resolution_10/xx_642_ch2/naruto/not
/Users/emilydennis/Desktop/test/rawdata/resolution_10/xx_642_ch2/naruto/other
/Users/emilydennis/Desktop/test/rawdata/resolution_10/xx_642_ch2/naruto/good
/Users/emilydennis/Desktop/test/rawdata/resolution_10/ch1_488_xx/killerbee
/Users/emilydennis/Desktop/test/rawdata/resoluti